In [1]:
import sys,os
sys.path.insert(0,'/global/project/projectdirs/metatlas/anaconda/lib/python2.7/site-packages' )
from metatlas import metatlas_objects as metob
import csv


curr_ld_lib_path = ''

os.environ['LD_LIBRARY_PATH'] = curr_ld_lib_path + ':/project/projectdirs/openmsi/jupyterhub_libs/boost_1_55_0/lib' + ':/project/projectdirs/openmsi/jupyterhub_libs/lib'
import sys
# sys.path.remove('/anaconda/lib/python2.7/site-packages')
sys.path.append('/global/project/projectdirs/openmsi/jupyterhub_libs/anaconda/lib/python2.7/site-packages')
sys.path.insert(0,'/project/projectdirs/openmsi/projects/meta-iq/pactolus/pactolus' )

from rdkit import Chem
# from rdkit.Chem.rdMolDescriptors import ExactMolWt
from rdkit.Chem import Descriptors
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem import AllChem
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit.Chem import Draw

from rdkit import DataStructs
import numpy as np

from matplotlib import pylab as plt

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import networkx as nx



The history saving thread hit an unexpected error (OperationalError('disk I/O error',)).History will not be written to the database.


In [2]:
# import pickle
# with open('save_compound_data_12.pkl', 'rb') as handle:
#     compoundData = pickle.load(handle)

In [2]:
allCompounds = metob.retrieve('Compounds',InchI='InchI%')
print len(allCompounds)

15479


In [3]:
# for c in allCompounds:
#     if 'AMP' in c.name:
#         print c

In [3]:
allCompounds[0]

{'InChI': u'InChI=1S/C15H24/c1-11(2)13-8-9-14-7-5-6-12(3)15(14,4)10-13/h7,12-13H,1,5-6,8-10H2,2-4H3/t12-,13-,15-/m1/s1',
 'MonoIsotopic_molecular_weight': 204.18780517578125,
 'creation_time': '2015-10-08T17:37:27',
 'description': u'',
 'formula': u'C15H24',
 'functional_sets': [],
 'head_id': u'0005ec9d4a8e4f1c9bd7cdee585a1ebc',
 'last_modified': '2015-10-08T17:38:13',
 'name': u'(-)-4-epieremophilene',
 'prev_uid': u'origin',
 'reference_xrefs': [],
 'synonyms': u'',
 'unique_id': u'0005ec9d4a8e4f1c9bd7cdee585a1ebc',
 'url': u'',
 'username': u'bpb'}

In [4]:
compoundData = {}
compoundData['myName'] = []
compoundData['myMW'] = []
compoundData['myFormula'] = []
compoundData['myID'] = []
compoundData['myMol'] = []
compoundData['myFingerprint'] = []
compoundData['inchi'] = []
compoundData['inchikey'] = []

N = 2048

for i,c in enumerate(allCompounds):
    if c.name != None:
        m = Chem.MolFromInchi(str(c.inchi))
        f = FingerprintMols.FingerprintMol(m,
                                     minPath=1,
                                     maxPath=7,
                                     fpSize=N,
                                     bitsPerHash=2,
                                     useHs=False,
                                     tgtDensity=0,
                                     minSize=N,
                                     branchedPaths=True,
                                     useBondOrder=True,
                                     atomInvariants=[],
                                     fromAtoms=[],
                                     atomBits=[])
        if np.sum(f) > 0:
            compoundData['myFingerprint'].append(f)
            compoundData['myMol'].append(m)
            compoundData['myName'].append(c.name)
            compoundData['myMW'].append(c.mono_isotopic_molecular_weight)
            compoundData['myFormula'].append(c.formula)
            compoundData['myID'].append(c.unique_id)
print len(compoundData['myFingerprint'])

15395


In [5]:
compoundData['fingerprint_dists'] = np.zeros((len(compoundData['myFingerprint']),len(compoundData['myFingerprint'])))
for i,fps1 in enumerate(compoundData['myFingerprint']):
    for j,fps2 in enumerate(compoundData['myFingerprint']):
        if i<j:
            compoundData['fingerprint_dists'][i,j] = DataStructs.FingerprintSimilarity(fps1,fps2)

compoundData['fingerprint_mat'] = np.zeros((len(compoundData['myFingerprint']),N))
# norm_fingerprint_mat = np.zeros((len(compoundData['myFingerprint']),N))
for i,f in enumerate(compoundData['myFingerprint']):
    for j in range(N):
        compoundData['fingerprint_mat'][i,j] = float(DataStructs.BitVectToText(compoundData['myFingerprint'][i])[j])

import pickle
with open('save_compound_data_0_no_Hs.pkl', 'wb') as handle:
    pickle.dump(compoundData, handle)

In [7]:
clusterData = {}

# print compoundData['fingerprint_dists'][:4,:4]
d = np.triu_indices(compoundData['fingerprint_dists'].shape[0],
                    1,
                    compoundData['fingerprint_dists'].shape[1])

vals = np.zeros(len(d[0]))
for i in range(len(d[0])):
    vals[i] = 1.0 - compoundData['fingerprint_dists'][d[0][i],d[1][i]]

# from scipy.spatial.distance import squareform
clusterData['distance'] = vals

In [11]:
1+1

2

In [8]:
with open('save_cluster_data_all_no_Hs.pkl', 'wb') as handle:
    pickle.dump(clusterData, handle)

In [3]:
import pickle
with open('save_cluster_data_all_no_Hs.pkl', 'r') as handle:
    clusterData = pickle.load(handle)

In [4]:


# print clusterData['distance'][:4,:4]

from scipy.cluster.hierarchy import linkage
clusterData['linkage'] = linkage(clusterData['distance'], method='average')


with open('save_cluster_data_all_no_Hs.pkl', 'wb') as handle:
    pickle.dump(clusterData, handle)

In [12]:
%system ls *.pkl

['save_cluster_data_0_no_Hs.pkl',
 'save_cluster_data_12.pkl',
 'save_cluster_data.pkl',
 'save_compound_data_0_no_Hs.pkl',
 'save_compound_data_12.pkl',
 'save_compound_data.pkl']

In [54]:
%%javascript
var nb = IPython.notebook;
var kernel = IPython.notebook.kernel;
var command = "NOTEBOOK_FULL_PATH = '" + nb.base_url + nb.notebook_path + "'";
kernel.execute(command);

<IPython.core.display.Javascript object>

In [58]:
filename = os.path.basename(NOTEBOOK_FULL_PATH)
%system cp $filename /project/projectdirs/openmsi/www/
temp = '%s/%s'%('/project/projectdirs/openmsi/www',filename)
%system chmod 775 $temp
print 'http://nbviewer.ipython.org/url/portal.nersc.gov/project/openmsi/%s?flush_cache=true'%filename

http://nbviewer.ipython.org/url/portal.nersc.gov/project/openmsi/Make_Network_of_MetAtlas_Compounds.ipynb?flush_cache=true
